# Question 3: Your own personal Netflix
In the lecture, we have briefly discussed a strategy for recommender systems to cope with the effect of many missing values in the rating matrix. The strategy was to minimize the approximation error only for the observed entries. That is, the optimization objective is $$\min_{X,Y} || D - O \circ (Y X^\intercal) ||^2 = \sum_{(i,k) \in \mathcal{O}} (D_{ik} - Y_{i.}X_{k.}^\intercal)^2 \text{ s.t. } X \in \mathbb{R}^{d \times r},Y \in \mathbb{R}^{n \times r}$$
where the matrix $O \in \{0,1\}^{n \times d}$ indicates the observed entries and the set $\mathcal{O} \subseteq \{1,...,n\} \times \{1, ... , d\}$ contains the indices of the observed entries in $D$. Note that the objective above is formulated a bit different from the one stated in the lecture. That is because we assume here that the matrix $D$ has an entry of zero if the corresponding value is missing. In this case, we have $D\circ O=D$ ( $\circ$ is the Hadamard multiplication, that is an element-wise multiplication of matrices) and we can write the objective to minimize subject to the observed entries as above. 
However, we have not discussed in detail how to optimize this objective. This will be done in this exercise.
The idea is to apply block coordinate descent on the rows of the factor matrices of $X$ and $Y$. We can obtain the global minimizers of the objective with respect to one row of $X$ or $Y$ (fixing all other coordinates) by means of FONC. First, we compute the partial gradient of the objective with respect to coordinate $X_{ks}$:
$$ \frac{\partial}{\partial X_{ks}} \sum_{(i,l) \in \mathcal{O}} (D_{il} - Y_{i.}X_{l.}^\intercal)^2 = 2 \sum_{(i,k) \in \mathcal{O}} (D_{ik} - Y_{i.} X_{k.}^\intercal)(-Y_{is}) = -2 (D_{.k} - YX_{k.}^\intercal)^\intercal \text{diag}(O_{.k})Y_{.s}$$
The matrix $O_{Xk}=\text{diag}(O_{\cdot k})$  is the diagonal matrix that has the indicator vector of observed entries of feature $k$ on the diagonal. Therewith, we can denote the gradient with respect to one row of $X$ as follows:
$$\nabla_{X_{k.}} \sum_{(i,l) \in \mathcal{O}} (D_{il} - Y_{i.}X_{l.}^\intercal)^2 = -2 (D_{.k}^\intercal - X_{k.}Y^\intercal)O_{Xk}Y$$
We compute now the stationary points of the gradient:
$$-2 (D_{.k}^\intercal - X_{k.}Y^\intercal)_{.k}O_{Xk}Y = 0 \iff D_{.k}^\intercal Y (Y^\intercal O_{Xk} Y)^{-1} = X_{k.}Y$$
Now we have here a small problem, since the matrix $Y^\intercal O_{k.} Y$ might not have an inverse. We have already seen how to alleviate this problem in the regression regularization lecture. Likewise, we add here a penalization term to the objective and obtain the penalized objective
$$\min_{X,Y} || D - O \circ (Y X^\intercal) ||^2 + \lambda ||X||^2 + \lambda ||Y||^2 \text{ s.t. } X \in \mathbb{R}^{d \times r}, Y \in \mathbb{R}^{n \times r}$$
The penalized objective has now well-defined stationary points $$D_{.k}^\intercal Y (Y^\intercal O_{Xk} Y + \lambda I)^{-1} = X_{k.}$$
Likewise, we can compute the minimizers of the objective for a row of $Y$, resulting in the following block-coordinate descent method for matrix completion.

**function** MatrixCompletion($D,r,t_{max} = 100,\lambda = 0.1$)
1. $(X,Y) \rightarrow$ InitRandom($n,d,r$)
2. $O \rightarrow$ IndicatorNonzero($D$)
3. **for** $t = 1$ **to** $t_{max}$
    1. **for** $k = 1$ **to** $d$
        1. $O_{Xk} \rightarrow$ diag($O_{1k}, ..., O_{nk}$)
        2. $X_{k.} \rightarrow D_{.k}^\intercal Y (Y^\intercal O_{Xk} Y + \lambda I)^{-1}$
    2. **for** $i = 1$ **to** $n$ 
        1. $O_{Yi} \rightarrow$ diag($O_{i1}, ..., O_{id}$)
        2. $Y_{i.} \rightarrow D_{i.} X (X^\intercal O_{Yi} X + \lambda I)^{-1}$
4. **return** $(X,Y)$

In the [Canvas material for this exercise](HW2_2IIGO_24_25.ipynb) is a small movie-lens dataset. You will find predefined implementations in the accompanying notebook to create a data matrix $D$, reflecting a small subset of the users and movies. Unless stated otherwise, we use as default parameter setting $t_{max}=100,r=20,\lambda=0.1$.

## 3a
Implement the  MatrixCompletion function and apply it to the movielens
data matrix $D$.
What is the average squared approximation error on the observed entries after 100 iterations? Round your result here at least to three decimals. (Note: $|O|$ is the sum of ones in $O$)

In [1]:
import numpy as np
import pandas as pd

In [2]:
print('The numpy version is {}.'.format(np.__version__))

The numpy version is 2.1.3.


In [3]:
# Implementation of the MatrixCompletion function
def matrix_completion(D, r = 20, t_max = 100, lmbda = 0.1):
    # Get the dimensions of the data matrix D
    n,d = D.shape
    # Initialize the factor matrices X and Y with random values
    np.random.seed(0)
    X = np.random.normal(size =(d,r))
    Y = np.random.normal(size =(n,r))
    # Create the indicator matrix O for observed entries
    O = np.where(D!=0,1,0)
    for t in range(t_max):
        # Update the rows of X
        for k in range(d):
            # Create the diagonal matrix O_Xk for the observed entries of column k
            O_Xk = np.diag(O[:,k])
            # Compute the matrix product Y^T O_Xk Y
            Y_O_Xk_Y = np.dot(np.dot(Y.T,O_Xk),Y) + lmbda*np.eye(r)
            # Update the k-th row of X
            X[k,:] = np.dot(np.dot(D[:,k].T,Y),np.linalg.inv(Y_O_Xk_Y))
        # Update the rows of Y    
        for i in range(n):
            # Create the diagonal matrix O_Yi for the observed entries of row i
            O_Yi = np.diag(O[i,:])
            # Compute the matrix product X^T O_Yi X
            X_O_Yi_X = np.dot(np.dot(X.T,O_Yi),X) + lmbda*np.eye(r)
            # Update the i-th row of Y
            Y[i,:] = np.dot(np.dot(D[i,:],X),np.linalg.inv(X_O_Yi_X))
    return X,Y

In [4]:
# Load and explore the movie-lens data
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Create the data matrix D
df_movie_ratings = ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)  #fill unobserved entries with μ

In [8]:
# Only keep movies that have been rated by at least 100 users
keep_movie = np.sum(df_movie_ratings!=0,0)>100
df_D = df_movie_ratings.loc[:,keep_movie]
df_D.head()

movieId,1,2,6,10,32,34,39,47,50,110,...,7153,7361,7438,8961,33794,48516,58559,60069,68954,79132
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,4.5,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Only keep users that have rated at least 5 movies
keep_user = np.sum(df_D!=0,1)>5
df_D = df_D.loc[keep_user,:]
df_D.head()

movieId,1,2,6,10,32,34,39,47,50,110,...,7153,7361,7438,8961,33794,48516,58559,60069,68954,79132
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,4.5,0.0,0.0,4.0
4,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,4.0,3.0,4.0,4.0,0.0,4.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Movie number - title mapping
movies.loc[movies['movieId'].isin(df_D.columns)]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
5,6,Heat (1995),Action|Crime|Thriller
9,10,GoldenEye (1995),Action|Adventure|Thriller
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
...,...,...,...
6315,48516,"Departed, The (2006)",Crime|Drama|Thriller
6710,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
6772,60069,WALL·E (2008),Adventure|Animation|Children|Romance|Sci-Fi
7039,68954,Up (2009),Adventure|Animation|Children|Drama


In [11]:
# Convert the dataframe to a data matrix
D = df_D.to_numpy()
D.shape

(547, 134)

In [12]:
# Apply the matrix completion function
X,Y = matrix_completion(D)

Average squared approximation error on the observed entries after 100 iterations: $$\frac{1}{|O|} || D - O \circ (Y X^\intercal) ||^2$$
Note that $|O|$ is the sum of ones in the indicator matrix $O$.

In [13]:
# Compute the average squared approximation error on the observed entries
O = np.where(D!=0,1,0)
error = np.linalg.norm((D - O*(np.dot(Y,X.T))))**2/np.sum(O)
error

np.float64(0.09442193783208083)

Indicate, for the following movies, the estimated rating of the first user.
* Finding Nemo (2003)
* Dark Knight, The (2008)
* Clueless (1995)
* 2001: A Space Odyssey (1968)

In [14]:
# Movie titles
movie_titles = movies.set_index('movieId')['title']
# find the movie ids from the titles
movie_ids = movie_titles[movie_titles.isin(['Finding Nemo (2003)',
                                             'Dark Knight, The (2008)',
                                             'Clueless (1995)',
                                             '2001: A Space Odyssey (1968)'])].index
# Print the movie ids and corresponding titles
movie_titles[movie_ids]

movieId
39                    Clueless (1995)
924      2001: A Space Odyssey (1968)
6377              Finding Nemo (2003)
58559         Dark Knight, The (2008)
Name: title, dtype: object

In [15]:
# Find the movie indices in the data matrix D
movie_indices = [np.where(df_D.columns==movie_id)[0][0] for movie_id in movie_ids]
movie_indices

[np.int64(6), np.int64(59), np.int64(121), np.int64(130)]

In [16]:
# Find the estimated ratings of the first user for the selected movies
user_id = 0
ratings = np.dot(Y[user_id,:],X.T)
ratings[movie_indices]

array([2.54167054, 4.31375458, 6.07512752, 6.45951606])

In [17]:
# Print the estimated ratings and corresponding movie titles
movie_titles[movie_ids].reset_index(drop=True).to_frame().assign(rating = ratings[movie_indices])

,title,rating
0,Clueless (1995),2.541671
1,2001: A Space Odyssey (1968),4.313755
2,Finding Nemo (2003),6.075128
3,"Dark Knight, The (2008)",6.459516


## 3b
Vary the value of the penalization parameter $\lambda \in \{0.01, 0.1, 0.5 \}$. Which effects do you observe?
- The higher $\lambda$ is, the lower the mean of the missing value imputations ($Y X^\intercal$ at the position where $O=0$) becomes.
- The higher $\lambda$ is, the lower the variance of the missing value imputations ($Y X^\intercal$ at the position where $O=0$) becomes.
- The higher $\lambda$ is, the fewer missing value imputations ($Y X^\intercal$ at the position where $O=0$) are outside the original range of ratings ($[0.5,5]$).
- The higher $\lambda$ is, the higher the approximation error $||D - O \circ (Y X^\intercal)||^2$ is.

In [18]:
# Compute the mean of the missing value imputations for different values of lambda
lambdas = [0.01,0.1,0.5]
means = []
for lmbda in lambdas:
    X,Y = matrix_completion(D,lmbda=lmbda)
    O = np.where(D!=0,1,0)
    # mean of the missing value imputations
    mean = np.sum((1-O)*(np.dot(Y,X.T)))/np.sum(1-O)
    means.append(mean)
means

[np.float64(3.3122244218026737),
 np.float64(3.034433215379759),
 np.float64(3.3269037928259566)]

In [19]:
# Compute the variance of the missing value imputations for different values of lambda
variances = []
for lmbda in lambdas:
    X,Y = matrix_completion(D,lmbda=lmbda)
    O = np.where(D!=0,1,0)
    # variance of the missing value imputations
    variance = np.sum((1-O)*(np.dot(Y,X.T) - np.sum((1-O)*(np.dot(Y,X.T)))/np.sum(1-O))**2)/np.sum(1-O)
    variances.append(variance)
variances

[np.float64(5.882683675525218),
 np.float64(3.3877693264990074),
 np.float64(1.3200977944734995)]

In [20]:
# Count the number of missing value imputations outside the original range of ratings for different values of lambda
counts = []
for lmbda in lambdas:
    X,Y = matrix_completion(D,lmbda=lmbda)
    O = np.where(D!=0,1,0)
    # number of missing value imputations outside the original range of ratings
    count = np.sum((1-O)*(np.dot(Y,X.T) < 0.5) + (1-O)*(np.dot(Y,X.T) > 5))
    counts.append(count)
counts

[np.int64(13788), np.int64(10124), np.int64(3861)]

In [21]:
# Compute the average squared approximation error on the observed entries for different values of lambda
lambdas = [0.01,0.1,0.5]
errors = []
for lmbda in lambdas:
    X,Y = matrix_completion(D,lmbda=lmbda)
    O = np.where(D!=0,1,0)
    # approximation error
    error = np.linalg.norm((D - O*(np.dot(Y,X.T))))**2
    errors.append(error)
errors

[np.float64(1815.1620853673728),
 np.float64(1855.2966564625563),
 np.float64(2032.037960977231)]